## Bias Mitigation using AIF360 - Heart Failure Prediction Dataset (Source: https://www.kaggle.com/datasets/fedesoriano/heart-failure-prediction/data)
Model Training and Evaluation

In [1]:
#load preprocessed data 
import pandas as pd
train_df = pd.read_csv("./data_subsets/train_50_50.csv")

X_test = pd.read_csv("./data_splits/X_test.csv")
y_test = pd.read_csv("./data_splits/y_test.csv")

#check out the data
train_df.head()

,Age,Sex,ChestPainType,RestingBP,Cholesterol,FastingBS,RestingECG,MaxHR,ExerciseAngina,Oldpeak,ST_Slope,HeartDisease
0,61,1,3,146.0,241.0,0,0,148.0,1,3.0,0,1
1,39,1,1,130.0,215.0,0,0,120.0,0,0.0,2,0
2,60,0,0,150.0,240.0,0,0,171.0,0,0.9,2,0
3,49,1,3,128.0,212.0,0,0,96.0,1,0.0,1,1
4,50,0,2,140.0,288.0,0,0,140.0,1,0.0,1,1


In [2]:
TARGET = "HeartDisease"
SENSITIVE = "Sex"   # 1 = Male, 0 = Female

categorical_cols = ['Sex','ChestPainType','FastingBS','RestingECG','ExerciseAngina','ST_Slope']
continuous_cols  = ['Age','RestingBP','Cholesterol','MaxHR','Oldpeak']

In [3]:
# Split train into X / y and keep sensitive feature for fairness evaluation
X_train = train_df.drop(columns=[TARGET])
y_train = train_df[TARGET]

In [4]:
# scale numeric features only, fit on train, transform test
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()

X_train_num_scaled = pd.DataFrame(
    scaler.fit_transform(X_train[continuous_cols]),
    columns=continuous_cols, index=X_train.index
)
X_test_num_scaled = pd.DataFrame(
    scaler.transform(X_test[continuous_cols]),
    columns=continuous_cols, index=X_test.index
)

In [5]:
#one-hot encode categoricals; numeric are kept as is 
from sklearn.preprocessing import OneHotEncoder

ohe = OneHotEncoder(handle_unknown="ignore", drop="if_binary", sparse_output=False)
ohe.fit(X_train[categorical_cols])

X_train_cat = pd.DataFrame(
    ohe.transform(X_train[categorical_cols]),
    columns=ohe.get_feature_names_out(categorical_cols),
    index=X_train.index
)
X_test_cat = pd.DataFrame(
    ohe.transform(X_test[categorical_cols]),
    columns=ohe.get_feature_names_out(categorical_cols),
    index=X_test.index
)

In [6]:
# Assemble final matrices
X_train_ready = pd.concat([X_train_cat, X_train_num_scaled], axis=1)
X_test_ready  = pd.concat([X_test_cat,  X_test_num_scaled],  axis=1)

print("Final feature shapes:", X_train_ready.shape, X_test_ready.shape)

Final feature shapes: (600, 18) (184, 18)


In [7]:
# create sensitive attribute arrays - after creating X_train_ready and X_test_ready
A_train = X_train["Sex"].astype(int).to_numpy().ravel()  # 1=Male, 0=Female
A_test  = X_test["Sex"].astype(int).to_numpy().ravel()

In [8]:
# setup for AIF360
import numpy as np, pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, recall_score, confusion_matrix
from sklearn.base import clone
from IPython.display import display 

from aif360.datasets import BinaryLabelDataset
from aif360.algorithms.preprocessing import Reweighing
from aif360.algorithms.postprocessing import EqOddsPostprocessing
from aif360.metrics import ClassificationMetric

# Config 
protected_attr = "Sex"  # 1=Male, 0=Female
PRIV_VALUE = 1          # privileged = Male
label_name = "label"
favorable_label, unfavorable_label = 1, 0
privileged_groups   = [{protected_attr: PRIV_VALUE}]
unprivileged_groups = [{protected_attr: 1 - PRIV_VALUE}]

# Ensure 1-D ints for targets
y_train = np.asarray(y_train).astype(int).ravel()
y_test  = np.asarray(y_test).astype(int).ravel()

# Sensitive attribute arrays
A_train = X_train["Sex"].astype(int).to_numpy().ravel()
A_test  = X_test["Sex"].astype(int).to_numpy().ravel()

def _to_bld(y, A):
    y = (y.values if hasattr(y,'values') else np.asarray(y)).ravel()
    A = (A.values if hasattr(A,'values') else np.asarray(A)).ravel()
    df = pd.DataFrame({"dummy": np.zeros(len(y)), label_name: y, protected_attr: A})
    return BinaryLabelDataset(
        df=df,
        label_names=[label_name],
        protected_attribute_names=[protected_attr],
        favorable_label=favorable_label,
        unfavorable_label=unfavorable_label
    )

def fair_metrics(y_true, y_pred, A, y_scores=None, absolute=True):
    """AIF360-based DP and EO (equal opportunity) differences."""
    t = _to_bld(y_true, A)
    p = _to_bld(y_pred, A)
    if y_scores is not None:
        p.scores = np.asarray(y_scores).reshape(-1, 1)
    cm = ClassificationMetric(
        t, p,
        privileged_groups=privileged_groups,
        unprivileged_groups=unprivileged_groups
    )
    dp = cm.statistical_parity_difference()
    eo = cm.equal_opportunity_difference()
    return (abs(dp), abs(eo)) if absolute else (dp, eo)

def get_scores(model, X):
    if hasattr(model, "predict_proba"):
        return model.predict_proba(X)[:, 1]
    if hasattr(model, "decision_function"):
        z = model.decision_function(X)
        return (z - z.min()) / (z.max() - z.min() + 1e-12)
    return model.predict(X).astype(float)

def selection_rate(y_pred, positive=1):
    y_pred = np.asarray(y_pred).ravel()
    return np.mean(y_pred == positive)

def per_group_table(y_true, y_pred, A, positive=1, group_name="Sex"):
    """Keeps your existing API (positive=...), uses sklearn metrics."""
    y_true = np.asarray(y_true).ravel()
    y_pred = np.asarray(y_pred).ravel()
    A = np.asarray(A).ravel()
    rows = []
    for g in np.unique(A):
        idx = (A == g)
        yt, yp = y_true[idx], y_pred[idx]
        tn, fp, fn, tp = confusion_matrix(yt, yp, labels=[0, 1]).ravel()
        tpr = tp / (tp + fn) if (tp + fn) else 0.0
        fpr = fp / (fp + tn) if (fp + tn) else 0.0
        rec = recall_score(yt, yp, pos_label=positive)   # equals TPR for binary
        sr  = selection_rate(yp, positive=positive)
        acc = accuracy_score(yt, yp)
        rows.append({group_name: g, "TPR": tpr, "FPR": fpr,
                     "Recall": rec, "SelectionRate": sr, "Accuracy": acc})
    return pd.DataFrame(rows).set_index(group_name)

def aif_diffs(y_true, y_pred, A, *, abs_vals=True):
    """Alternative disparities (AIF360): DP and average odds difference."""
    t = _to_bld(y_true, A)
    p = _to_bld(y_pred, A)
    cm = ClassificationMetric(
        t, p,
        privileged_groups=privileged_groups,
        unprivileged_groups=unprivileged_groups
    )
    dp = cm.statistical_parity_difference()
    eo = cm.average_odds_difference()   # avg of TPR/FPR diffs
    if abs_vals:
        dp, eo = abs(dp), abs(eo)
    return dp, eo

def print_row(title, acc, dp, eo, note=""):
    print(f"{title:>24s} | Acc {acc:.4f} | DP {dp:.4f} | EO {eo:.4f} {('|' if note else '')} {note}")

# to print a model cleanly (fixed call sites)
def report_model(name, y_true, y_pred, A, scores=None, note=""):
    acc = accuracy_score(y_true, y_pred)
    dp, eo = fair_metrics(y_true, y_pred, A, y_scores=scores, absolute=True)  # no pos_label here
    tbl = per_group_table(y_true, y_pred, A, positive=favorable_label, group_name="Sex").round(6)
    
    print(f"\n=== {name} ===")
    display(tbl)
    print(f"Overall -> Accuracy: {acc:.4f} | DP diff: {dp:.4f} | EO diff: {eo:.4f}"
          + (f" | {note}" if note else ""))
    
    return {"Model": name, "Accuracy": acc, "DP diff": dp, "EO diff": eo}

# Pre: compute reweighing weights ONCE on TRAIN
_bld_train = BinaryLabelDataset(
    df=pd.concat([pd.DataFrame(X_train_ready),
                  pd.Series(y_train, name=label_name),
                  pd.Series(A_train, name=protected_attr)], axis=1),
    label_names=[label_name],
    protected_attribute_names=[protected_attr],
    favorable_label=favorable_label,
    unfavorable_label=unfavorable_label
)
_rw = Reweighing(unprivileged_groups=unprivileged_groups,
                 privileged_groups=privileged_groups).fit(_bld_train)
_rw_weights = _rw.transform(_bld_train).instance_weights.ravel()

# Turn weights into a resampled training set
def resample_by_weights(X, y, A, weights, n_samples=None, random_state=42):
    rng = np.random.default_rng(random_state)
    Xn = np.asarray(X); yn = np.asarray(y).ravel(); An = np.asarray(A).ravel()
    w = np.clip(np.asarray(weights, dtype=float), 1e-12, None)
    p = w / w.sum()
    n = n_samples or len(yn)
    idx = rng.choice(len(yn), size=n, replace=True, p=p)
    return Xn[idx], yn[idx], An[idx]

Xrw, yrw, Arw = resample_by_weights(
    X_train_ready, y_train, A_train, _rw_weights,
    n_samples=len(y_train), random_state=42
)

# Post: make a small TRAIN-based calibration split (no test leakage)
trn_X, cal_X, trn_y, cal_y, trn_A, cal_A = train_test_split(
    X_train_ready, y_train, A_train, test_size=0.12, stratify=y_train, random_state=42
)

# Make types consistent to avoid the PCA warning 
X_test_np = np.asarray(X_test_ready)
trn_X_np  = np.asarray(trn_X)
cal_X_np  = np.asarray(cal_X)

C:\Users\patri\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\inFairness\utils\ndcg.py:37: FutureWarning: We've integrated functorch into PyTorch. As the final step of the integration, `functorch.vmap` is deprecated as of PyTorch 2.0 and will be deleted in a future version of PyTorch >= 2.3. Please use `torch.vmap` instead; see the PyTorch 2.0 release notes and/or the `torch.func` migration guide for more details https://pytorch.org/docs/main/func.migrating.html
  vect_normalized_discounted_cumulative_gain = vmap(
C:\Users\patri\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\inFairness\utils\ndcg.py:48: FutureWarning: We've integrated functorch into PyTorch. As the final step of the integration, `functorch.vmap` is deprecated as of PyTorch 2.0 and will be deleted in a future version of PyTorch >= 2.3. Please use `torch.vmap` instead; 

In [9]:
import warnings
warnings.filterwarnings("ignore", category=UserWarning, module="sklearn")

### Traditional ML Models - Baseline: K-Nearest Neighbors (KNN) & Decision Tree (DT)

In [10]:
#import required libraries
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import (
    accuracy_score, precision_score, recall_score, f1_score,
    classification_report, confusion_matrix
)

#define a function 
def evaluate_model(y_true, y_pred, model_name):
    print(f"=== {model_name} Evaluation ===")
    print("Accuracy :", accuracy_score(y_true, y_pred))
    print("Precision:", precision_score(y_true, y_pred, average='binary'))
    print("Recall   :", recall_score(y_true, y_pred, average='binary'))
    print("F1 Score :", f1_score(y_true, y_pred, average='binary'))
    print("\nClassification Report:\n", classification_report(y_true, y_pred))
    print("Confusion Matrix:\n", confusion_matrix(y_true, y_pred))
    print("\n" + "="*40 + "\n")

### PCA-KNN

In [11]:
from sklearn.decomposition import PCA
from sklearn.pipeline import Pipeline
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import (
    accuracy_score, precision_score, recall_score, f1_score,
    classification_report, confusion_matrix
)
import numpy as np

#1) PCA + KNN pipeline (on one-hot encoded + scaled features)
pca_knn = Pipeline([
    ('pca', PCA(n_components=0.95, random_state=42)),  # keep 95% variance
    ('knn', KNeighborsClassifier(
        n_neighbors=15, metric='manhattan', weights='distance'
    ))
])

pca_knn.fit(X_train_ready, y_train)

# Inspect PCA details
n_comp = pca_knn.named_steps['pca'].n_components_
expl_var = pca_knn.named_steps['pca'].explained_variance_ratio_.sum()
print(f"PCA components: {n_comp} | Explained variance retained: {expl_var:.3f}")

# 2) Evaluate 
y_pred_pca_knn = pca_knn.predict(X_test_ready)
probs_pca_knn = pca_knn.predict_proba(X_test_ready)[:, 1]
  
evaluate_model(y_test, y_pred_pca_knn, "KNN (best params)")

PCA components: 12 | Explained variance retained: 0.969
=== KNN (best params) Evaluation ===
Accuracy : 0.875
Precision: 0.9247311827956989
Recall   : 0.8431372549019608
F1 Score : 0.882051282051282

Classification Report:
               precision    recall  f1-score   support

           0       0.82      0.91      0.87        82
           1       0.92      0.84      0.88       102

    accuracy                           0.88       184
   macro avg       0.87      0.88      0.87       184
weighted avg       0.88      0.88      0.88       184

Confusion Matrix:
 [[75  7]
 [16 86]]




### Tuned Decision Tree (DT)

In [12]:
# Alternative DT tuning: simpler trees + class balancing + cost-complexity pruning
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV, StratifiedKFold, cross_val_score
from sklearn.metrics import (
    accuracy_score, precision_score, recall_score, f1_score,
    classification_report, confusion_matrix
)
import numpy as np

# bias toward simpler trees with class_weight="balanced" 
base_dt = DecisionTreeClassifier(random_state=42, class_weight="balanced")

param_grid_simple = {
    "criterion": ["gini", "entropy"],
    "max_depth": [3, 4, 5, 6, 7],
    "min_samples_split": [5, 10, 20],
    "min_samples_leaf": [2, 4, 6],
    "min_impurity_decrease": [0.0, 1e-4, 1e-3],  # tiny regularization
}

cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

grid_simple = GridSearchCV(
    estimator=base_dt,
    param_grid=param_grid_simple,
    cv=cv,
    scoring="recall",    # balanced focus
    n_jobs=-1,
    verbose=0,
    refit=True
)
grid_simple.fit(X_train_ready, y_train)

print("Stage A — Best simple DT params:", grid_simple.best_params_)
print("Stage A — Best CV F1:", grid_simple.best_score_)
simple_dt = grid_simple.best_estimator_

# cost-complexity pruning
path = simple_dt.cost_complexity_pruning_path(X_train_ready, y_train)
ccp_alphas = path.ccp_alphas

unique_alphas = np.unique(np.round(ccp_alphas, 6))
candidate_alphas = np.linspace(unique_alphas.min(), unique_alphas.max(), num=min(20, len(unique_alphas)))
candidate_alphas = np.unique(np.concatenate([candidate_alphas, [0.0]]))  

cv_scores = []
for alpha in candidate_alphas:
    dt_alpha = DecisionTreeClassifier(
        random_state=42,
        class_weight="balanced",
        criterion=simple_dt.criterion,
        max_depth=simple_dt.max_depth,
        min_samples_split=simple_dt.min_samples_split,
        min_samples_leaf=simple_dt.min_samples_leaf,
        min_impurity_decrease=simple_dt.min_impurity_decrease,
        ccp_alpha=alpha
    )
    f1_cv = cross_val_score(dt_alpha, X_train_ready, y_train, cv=cv, scoring="recall", n_jobs=-1).mean()
    cv_scores.append((alpha, f1_cv))

best_alpha, best_cv_f1 = sorted(cv_scores, key=lambda x: x[1], reverse=True)[0]
print(f"Stage B — Best ccp_alpha: {best_alpha:.6f} | CV F1: {best_cv_f1:.4f}")

best_dt = DecisionTreeClassifier(
    random_state=42,
    class_weight="balanced",
    criterion=simple_dt.criterion,
    max_depth=simple_dt.max_depth,
    min_samples_split=simple_dt.min_samples_split,
    min_samples_leaf=simple_dt.min_samples_leaf,
    min_impurity_decrease=simple_dt.min_impurity_decrease,
    ccp_alpha=best_alpha
).fit(X_train_ready, y_train)

# Evaluate on test set 
y_pred_dt = best_dt.predict(X_test_ready)
y_prob_dt = best_dt.predict_proba(X_test_ready)[:, 1]  

evaluate_model(y_test, y_pred_dt, "Alternative Tuned & Pruned Decision Tree")

Stage A — Best simple DT params: {'criterion': 'entropy', 'max_depth': 6, 'min_impurity_decrease': 0.0, 'min_samples_leaf': 4, 'min_samples_split': 5}
Stage A — Best CV F1: 0.8800000000000001
Stage B — Best ccp_alpha: 0.000000 | CV F1: 0.8800
=== Alternative Tuned & Pruned Decision Tree Evaluation ===
Accuracy : 0.8206521739130435
Precision: 0.896551724137931
Recall   : 0.7647058823529411
F1 Score : 0.8253968253968254

Classification Report:
               precision    recall  f1-score   support

           0       0.75      0.89      0.82        82
           1       0.90      0.76      0.83       102

    accuracy                           0.82       184
   macro avg       0.82      0.83      0.82       184
weighted avg       0.83      0.82      0.82       184

Confusion Matrix:
 [[73  9]
 [24 78]]




### Ensemble Model - Random Forest (RF)

In [13]:
from sklearn.ensemble import RandomForestClassifier

# Initialize Random Forest
rf = RandomForestClassifier(random_state=42)

# Train the model
rf.fit(X_train_ready, y_train)

# Predict on test set
y_pred_rf = rf.predict(X_test_ready)
y_prob_rf = rf.predict_proba(X_test_ready)[:, 1]  
evaluate_model(y_test, y_pred_rf, "Random Forest")

=== Random Forest Evaluation ===
Accuracy : 0.8315217391304348
Precision: 0.8585858585858586
Recall   : 0.8333333333333334
F1 Score : 0.845771144278607

Classification Report:
               precision    recall  f1-score   support

           0       0.80      0.83      0.81        82
           1       0.86      0.83      0.85       102

    accuracy                           0.83       184
   macro avg       0.83      0.83      0.83       184
weighted avg       0.83      0.83      0.83       184

Confusion Matrix:
 [[68 14]
 [17 85]]




### Deep Learning - Multi-layer Perceptron

In [14]:
#import required library 
from sklearn.neural_network import MLPClassifier

### Recall-First tuned MLP

In [15]:
#  Improved MLP pipeline: recall-first tuning  
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import RandomizedSearchCV, StratifiedKFold
from sklearn.metrics import (
    f1_score, recall_score, fbeta_score, make_scorer
)


# 1) Recall-first search (Adam + early_stopping)
base_mlp = MLPClassifier(
    solver="adam",
    early_stopping=True,          # uses internal 15% validation
    validation_fraction=0.15,
    n_iter_no_change=20,
    max_iter=2000,                # allow convergence
    random_state=42
)

param_dist = {
    "hidden_layer_sizes": [(64,), (128,), (64, 32), (128, 64)],
    "activation": ["relu", "tanh"],
    "alpha": [1e-5, 1e-4, 3e-4, 1e-3],
    "learning_rate_init": [1e-3, 5e-4, 3e-4, 1e-4],
    "batch_size": [16, 32, 64],
}

cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

# multi-metric scoring; refit on recall-oriented F-beta
scoring = {
    "f1": make_scorer(f1_score),
    "recall": make_scorer(recall_score),
    "fbeta2": make_scorer(fbeta_score, beta=2)  # emphasize recall
}

rs = RandomizedSearchCV(
    estimator=base_mlp,
    param_distributions=param_dist,
    n_iter=30,
    scoring=scoring,
    refit="fbeta2",
    cv=cv,
    n_jobs=-1,
    verbose=1,
    random_state=42
)

rs.fit(X_train_ready, y_train)
recallfirst_best_mlp = rs.best_estimator_
print("Best MLP params:", rs.best_params_)

# 2) Evaluation
y_prob = recallfirst_best_mlp.predict_proba(X_test_ready)[:, 1]
y_pred_best_mlp = recallfirst_best_mlp.predict(X_test_ready)
evaluate_model(y_test, y_pred_best_mlp, model_name=f"Best MLP")

Fitting 5 folds for each of 30 candidates, totalling 150 fits
Best MLP params: {'learning_rate_init': 0.001, 'hidden_layer_sizes': (64, 32), 'batch_size': 16, 'alpha': 0.001, 'activation': 'relu'}
=== Best MLP Evaluation ===
Accuracy : 0.8532608695652174
Precision: 0.9120879120879121
Recall   : 0.8137254901960784
F1 Score : 0.8601036269430051

Classification Report:
               precision    recall  f1-score   support

           0       0.80      0.90      0.85        82
           1       0.91      0.81      0.86       102

    accuracy                           0.85       184
   macro avg       0.85      0.86      0.85       184
weighted avg       0.86      0.85      0.85       184

Confusion Matrix:
 [[74  8]
 [19 83]]




### Bias Mitigation AIF360

In [16]:
# Setup: install AIF360
# Uncomment the next line if running locally for the first time
#!pip install aif360

In [17]:
import aif360
print("AIF360 version:", aif360.__version__)

AIF360 version: 0.6.1


### Bias Mitigation AIF 360 - KNN 

In [18]:
#get the Fairlearn KNN Baseline for AIF360 bias mitigation
knn_base = pca_knn

yhat_knn_base   = knn_base.predict(X_test_ready)         
scores_knn_base = get_scores(knn_base, X_test_ready)

res_knn_base = report_model("Fairlearn - KNN baseline", y_test, yhat_knn_base, A_test, scores=scores_knn_base)


#Pre (Reweighing)
knn_pre        = clone(pca_knn).fit(Xrw, yrw)
yhat_knn_pre   = knn_pre.predict(X_test_ready)
scores_knn_pre = get_scores(knn_pre, X_test_ready)
res_knn_pre    = report_model("KNN pre: Reweigh",
                              y_test, yhat_knn_pre, A_test,
                              scores=scores_knn_pre,
                              note="resampled by AIF360 weights")

#Post (Equalized Odds)
cal_scores_knn   = get_scores(knn_base, cal_X_np)  # baseline KNN on CAL
post_knn = EqOddsPostprocessing(privileged_groups=privileged_groups,
                                unprivileged_groups=unprivileged_groups)
post_knn.fit(_to_bld(cal_y, cal_A),
             _to_bld((cal_scores_knn >= 0.5).astype(int), cal_A))

pred_knn_post_bld = post_knn.predict(_to_bld((scores_knn_base >= 0.5).astype(int), A_test))
yhat_knn_post     = pred_knn_post_bld.labels.ravel().astype(int)

res_knn_post = report_model("KNN post: EqOdds",
                            y_test, yhat_knn_post, A_test,
                            scores=scores_knn_base,
                            note="calibrated on held-out TRAIN")


=== Fairlearn - KNN baseline ===


,TPR,FPR,Recall,SelectionRate,Accuracy
Sex,,,,,
0,0.666667,0.0625,0.666667,0.157895,0.894737
1,0.854167,0.1000,0.854167,0.595890,0.869863


Overall -> Accuracy: 0.8750 | DP diff: 0.4380 | EO diff: 0.1875

=== KNN pre: Reweigh ===


,TPR,FPR,Recall,SelectionRate,Accuracy
Sex,,,,,
0,0.833333,0.09375,0.833333,0.210526,0.894737
1,0.791667,0.14000,0.791667,0.568493,0.815068


Overall -> Accuracy: 0.8315 | DP diff: 0.3580 | EO diff: 0.0417 | resampled by AIF360 weights

=== KNN post: EqOdds ===


,TPR,FPR,Recall,SelectionRate,Accuracy
Sex,,,,,
0,0.666667,0.0625,0.666667,0.157895,0.894737
1,0.854167,0.1000,0.854167,0.595890,0.869863


Overall -> Accuracy: 0.8750 | DP diff: 0.4380 | EO diff: 0.1875 | calibrated on held-out TRAIN


## KNN + AIF360

### Results overview
| Variant             | Accuracy | DP diff | EO diff (TPR gap) | DP+EO |
|---------------------|---------:|--------:|-------------------:|------:|
| Baseline            | 0.8750   | 0.4380  | 0.1875             | 0.6255 |
| Pre: Reweigh        | 0.8315   | 0.3580  | **0.0417**         | **0.3997** |
| Post: EqualizedOdds | 0.8750   | 0.4380  | 0.1875             | 0.6255 |

---

### Per-group behavior (Female → 0, Male → 1)

#### Baseline
- **Selection rate:** 0 **0.158**, 1 **0.596** → large selection gap (**DP 0.438**).
- **TPR (Recall):** 0 **0.667**, 1 **0.854** → recall gap (**EO 0.188**), males detected more often.
- **FPR:** 0 **0.063**, 1 **0.100**.
- **Note:** Good accuracy but fairness gaps remain significant.

#### Pre-processing: Reweigh
- **Selection rate:** 0 **0.211**, 1 **0.568** → DP improves to **0.358**.
- **TPR (Recall):** 0 **0.833**, 1 **0.792** → **near-parity recall** (**EO 0.042**, best).
- **FPR:** 0 **0.094**, 1 **0.140** (both ↑ compared to baseline).
- **Note:** Best fairness (lowest DP+EO), with some accuracy drop.

#### Post-processing: Equalized Odds
- **Selection rate:** 0 **0.158**, 1 **0.596** → DP **0.438** (same as baseline).
- **TPR (Recall):** 0 **0.667**, 1 **0.854** → recall gap unchanged (**EO 0.188**).
- **FPR:** 0 **0.063**, 1 **0.100** (same as baseline).
- **Note:** Essentially identical to baseline (calibrated on train), no real fairness gains.

---

### Implications
- **Most fair overall:** **Pre: Reweigh**, by nearly eliminating the recall gap and shrinking DP, at the cost of some accuracy.
- **Post: EqOdds** failed to improve over baseline in this setup — fairness metrics remained unchanged.
- **Baseline** retains decent accuracy but shows strong disparities in both selection and recall.

---

In [19]:
#get the Fairlearn DT Baseline for AIF360 bias mitigation
dt_base = best_dt

yhat_dt_base   = dt_base.predict(X_test_ready)         
scores_dt_base = get_scores(dt_base, X_test_ready)

res_dt_base = report_model("Fairlearn - DT baseline", y_test, yhat_dt_base, A_test, scores=scores_dt_base)

# Pre (Reweighing)
dt_pre = clone(best_dt).fit(Xrw, yrw)
yhat_dt_pre = dt_pre.predict(X_test_np)
scores_dt_pre = get_scores(dt_pre, X_test_np)
_ = report_model("DT pre: Reweigh", y_test, yhat_dt_pre, A_test, scores=scores_dt_pre,
                 note="resampled by AIF360 weights")

# Post (Equalized Odds)
cal_scores_dt = get_scores(dt_base, cal_X_np)
post_dt = EqOddsPostprocessing(privileged_groups=privileged_groups,
                               unprivileged_groups=unprivileged_groups)
post_dt.fit(_to_bld(cal_y, cal_A),
            _to_bld((cal_scores_dt >= 0.5).astype(int), cal_A))
yhat_dt_post = post_dt.predict(_to_bld((scores_dt_base >= 0.5).astype(int), A_test)).labels.ravel().astype(int)
_ = report_model("DT post: EqOdds", y_test, yhat_dt_post, A_test, scores=scores_dt_base,
                 note="calibrated on held-out TRAIN")


=== Fairlearn - DT baseline ===


,TPR,FPR,Recall,SelectionRate,Accuracy
Sex,,,,,
0,0.666667,0.125,0.666667,0.210526,0.842105
1,0.770833,0.100,0.770833,0.541096,0.815068


Overall -> Accuracy: 0.8207 | DP diff: 0.3306 | EO diff: 0.1042

=== DT pre: Reweigh ===


,TPR,FPR,Recall,SelectionRate,Accuracy
Sex,,,,,
0,0.833333,0.1875,0.833333,0.289474,0.815789
1,0.656250,0.1600,0.656250,0.486301,0.719178


Overall -> Accuracy: 0.7391 | DP diff: 0.1968 | EO diff: 0.1771 | resampled by AIF360 weights

=== DT post: EqOdds ===


,TPR,FPR,Recall,SelectionRate,Accuracy
Sex,,,,,
0,0.666667,0.21875,0.666667,0.289474,0.763158
1,0.833333,0.14000,0.833333,0.595890,0.842466


Overall -> Accuracy: 0.8261 | DP diff: 0.3064 | EO diff: 0.1667 | calibrated on held-out TRAIN


## DT + AIF360

### Results overview
| Variant             | Accuracy | DP diff | EO diff (TPR gap) | DP+EO |
|---------------------|---------:|--------:|-------------------:|------:|
| Baseline            | 0.8207   | 0.3306  | **0.1042**         | **0.4348** |
| Pre: Reweigh        | 0.7391   | **0.1968** | 0.1771          | 0.3739 |
| Post: EqualizedOdds | 0.8261   | 0.3064  | 0.1667             | 0.4731 |

---

### Per-group reading (Female → 0, Male → 1)

#### Baseline
- **Selection rate:** 0 **0.211**, 1 **0.541** → DP gap **0.331**.
- **TPR (Recall):** 0 **0.667**, 1 **0.771** → EO gap **0.104** (small).
- **FPR:** 0 **0.125**, 1 **0.100**.
- **Note:** Balanced performance with relatively small recall gap, but notable DP disparity.

#### Pre-processing: Reweigh
- **Selection rate:** 0 **0.289**, 1 **0.486** → DP improves to **0.197**.
- **TPR (Recall):** 0 **0.833**, 1 **0.656** → recall gap grows (**EO 0.177**).
- **FPR:** 0 **0.188** (↑), 1 **0.160** (↑).
- **Note:** Best DP, but worsens EO and overall accuracy.

#### Post-processing: Equalized Odds
- **Selection rate:** 0 **0.289**, 1 **0.596** → DP **0.306** (better than baseline, worse than pre).
- **TPR (Recall):** 0 **0.667**, 1 **0.833** → EO gap **0.167**.
- **FPR:** 0 **0.219** (↑), 1 **0.140** (↑).
- **Note:** Accuracy slightly above baseline; EO worse; DP moderately improved.

---

### Implications
- **Most balanced:** **Baseline** (smallest EO and decent accuracy, though DP gap remains high).
- **Best DP:** **Pre: Reweigh**, but this creates a bigger recall gap and reduces accuracy.
- **Highest accuracy:** **Post: EqOdds**, but fairness benefits are limited (EO worsens, DP only partly improved).

---

In [20]:
# Random Forest (RF) with AIF360

# -get Fairlearn baseline
yhat_rf_base    = rf.predict(X_test_ready)
scores_rf_base  = get_scores(rf, X_test_ready)
res_rf_base     = report_model("Fairlearn - RF baseline", y_test, yhat_rf_base, A_test, scores=scores_rf_base)

# Pre (Reweighing via sample_weight)
rf_pre          = clone(rf).fit(X_train_ready, y_train, sample_weight=_rw_weights)
yhat_rf_pre     = rf_pre.predict(X_test_ready)
scores_rf_pre   = get_scores(rf_pre, X_test_ready)
res_rf_pre      = report_model("RF pre: Reweigh (sample_weight)",
                               y_test, yhat_rf_pre, A_test, scores=scores_rf_pre)

# Post (Equalized Odds) learned on CAL
cal_scores_rf   = get_scores(rf, cal_X_np)  # baseline rf on calibration split
post_rf = EqOddsPostprocessing(privileged_groups=privileged_groups,
                               unprivileged_groups=unprivileged_groups)
post_rf.fit(_to_bld(cal_y, cal_A),
            _to_bld((cal_scores_rf >= 0.5).astype(int), cal_A))

pred_rf_post_bld = post_rf.predict(_to_bld((scores_rf_base >= 0.5).astype(int), A_test))
yhat_rf_post     = pred_rf_post_bld.labels.ravel().astype(int)

res_rf_post = report_model("RF post: EqOdds",
                           y_test, yhat_rf_post, A_test,
                           scores=scores_rf_base,
                           note="calibrated on held-out TRAIN")



=== Fairlearn - RF baseline ===


,TPR,FPR,Recall,SelectionRate,Accuracy
Sex,,,,,
0,0.833333,0.15625,0.833333,0.263158,0.842105
1,0.833333,0.18000,0.833333,0.609589,0.828767


Overall -> Accuracy: 0.8315 | DP diff: 0.3464 | EO diff: 0.0000

=== RF pre: Reweigh (sample_weight) ===


,TPR,FPR,Recall,SelectionRate,Accuracy
Sex,,,,,
0,0.833333,0.15625,0.833333,0.263158,0.842105
1,0.833333,0.18000,0.833333,0.609589,0.828767


Overall -> Accuracy: 0.8315 | DP diff: 0.3464 | EO diff: 0.0000

=== RF post: EqOdds ===


,TPR,FPR,Recall,SelectionRate,Accuracy
Sex,,,,,
0,0.833333,0.15625,0.833333,0.263158,0.842105
1,0.833333,0.20000,0.833333,0.616438,0.821918


Overall -> Accuracy: 0.8261 | DP diff: 0.3533 | EO diff: 0.0000 | calibrated on held-out TRAIN


## RF + AIF360 

### Results overview
| Variant             | Accuracy | DP diff | EO diff (TPR gap) | DP+EO |
|---------------------|---------:|--------:|-------------------:|------:|
| Baseline            | 0.8315   | 0.3464  | **0.0000**         | **0.3464** |
| Pre: Reweigh        | 0.8315   | 0.3464  | **0.0000**         | **0.3464** |
| Post: EqualizedOdds | 0.8261   | 0.3533  | **0.0000**         | 0.3533 |

---

### Per-group behavior (Female → 0, Male → 1)

#### Baseline
- **Selection rate:** 0 **0.263**, 1 **0.610** → DP gap **0.346** (large).  
- **TPR (Recall):** 0 **0.833**, 1 **0.833** → **perfect recall parity** (EO **0.000**).  
- **FPR:** 0 **0.156**, 1 **0.180**.  
- **Note:** Accuracy 0.832, good EO, but large DP disparity.

#### Pre-processing: Reweigh
- **Selection rate:** 0 **0.263**, 1 **0.610** → identical DP **0.346**.  
- **TPR:** unchanged (both **0.833**).  
- **FPR:** unchanged.  
- **Note:** No effect — results are exactly the same as baseline with sample weighting.

#### Post-processing: Equalized Odds
- **Selection rate:** 0 **0.263**, 1 **0.616** → DP **0.353** (slightly worse than baseline).  
- **TPR:** still **parity (0.833 vs 0.833)** → EO **0.000**.  
- **FPR:** 0 **0.156**, 1 **0.200** (male ↑ slightly).  
- **Note:** Accuracy drops slightly (0.826 vs 0.832); fairness essentially unchanged, DP slightly worse.

---

### Implications
- **Most fair overall:** **Baseline** already achieves **perfect EO** with the lowest DP+EO (0.346).  
- **Pre: Reweigh** provides **no changes** in this setup.  
- **Post: EqOdds** slightly reduces accuracy and worsens DP, without improving fairness.  
- For RF, **baseline is already optimal** in terms of fairness–performance balance.

---

In [21]:
#get the Fairlearn MLP Baseline for AIF360 bias mitigation
mlp_base = recallfirst_best_mlp
yhat_mlp_base   = mlp_base.predict(X_test_ready)         
scores_mlp_base = get_scores(mlp_base, X_test_ready)

res_mlp_base = report_model("Fairlearn - MLP baseline", y_test, yhat_mlp_base, A_test, scores=scores_mlp_base)

# Pre (Reweighing)
mlp_pre = clone(recallfirst_best_mlp).fit(Xrw, yrw)
yhat_mlp_pre = mlp_pre.predict(X_test_np)
scores_mlp_pre = get_scores(mlp_pre, X_test_np)
_ = report_model("MLP pre: Reweigh", y_test, yhat_mlp_pre, A_test, scores=scores_mlp_pre,
                 note="resampled by AIF360 weights")

# Post (Equalized Odds)
cal_scores_mlp = get_scores(mlp_base, cal_X_np)
post_mlp = EqOddsPostprocessing(privileged_groups=privileged_groups,
                                unprivileged_groups=unprivileged_groups)
post_mlp.fit(_to_bld(cal_y, cal_A),
             _to_bld((cal_scores_mlp >= 0.5).astype(int), cal_A))
yhat_mlp_post = post_mlp.predict(_to_bld((scores_mlp_base >= 0.5).astype(int), A_test)).labels.ravel().astype(int)
_ = report_model("MLP post: EqOdds", y_test, yhat_mlp_post, A_test, scores=scores_mlp_base,
                 note="calibrated on held-out TRAIN")


=== Fairlearn - MLP baseline ===


,TPR,FPR,Recall,SelectionRate,Accuracy
Sex,,,,,
0,0.666667,0.0625,0.666667,0.157895,0.894737
1,0.822917,0.1200,0.822917,0.582192,0.842466


Overall -> Accuracy: 0.8533 | DP diff: 0.4243 | EO diff: 0.1562

=== MLP pre: Reweigh ===


,TPR,FPR,Recall,SelectionRate,Accuracy
Sex,,,,,
0,0.666667,0.09375,0.666667,0.184211,0.868421
1,0.802083,0.14000,0.802083,0.575342,0.821918


Overall -> Accuracy: 0.8315 | DP diff: 0.3911 | EO diff: 0.1354 | resampled by AIF360 weights

=== MLP post: EqOdds ===


,TPR,FPR,Recall,SelectionRate,Accuracy
Sex,,,,,
0,0.833333,0.125,0.833333,0.236842,0.868421
1,0.822917,0.120,0.822917,0.582192,0.842466


Overall -> Accuracy: 0.8478 | DP diff: 0.3453 | EO diff: 0.0104 | calibrated on held-out TRAIN


## MLP + AIF360

### Results overview
| Variant             | Accuracy | DP diff | EO diff (TPR gap) | DP+EO |
|---------------------|---------:|--------:|-------------------:|------:|
| Baseline            | 0.8533   | 0.4243  | 0.1562             | 0.5805 |
| Pre: Reweigh        | 0.8315   | 0.3911  | **0.1354**         | **0.5265** |
| Post: EqualizedOdds | 0.8370   | **0.3453** | 0.1562          | 0.5015 |

---

### Per-group behavior (Female → 0, Male → 1)

#### Baseline
- **Selection rate:** 0 **0.158**, 1 **0.582** → large DP (**0.424**).
- **TPR (Recall):** 0 **0.667**, 1 **0.823** → EO gap **0.156**.
- **FPR:** 0 **0.063**, 1 **0.120**.
- **Note:** Highest accuracy, but fairness disparities remain large.

#### Pre-processing: Reweigh
- **Selection rate:** 0 **0.184**, 1 **0.575** → DP improves to **0.391**.
- **TPR:** 0 **0.667**, 1 **0.802** → **best EO** (**0.135**).
- **FPR:** 0 **0.094**, 1 **0.140** (both ↑ compared to baseline).
- **Note:** Improves both DP and EO somewhat, at a small accuracy cost.

#### Post-processing: Equalized Odds
- **Selection rate:** 0 **0.237**, 1 **0.582** → **best DP** (**0.345**).
- **TPR:** 0 **0.667**, 1 **0.823** → EO gap same as baseline (**0.156**).
- **FPR:** 0 **0.156** (↑ sharply), 1 **0.120** (≈ baseline).
- **Note:** Strongest DP reduction, but through higher female FPR; accuracy in between baseline and reweigh.

---

### Implications
- **Most fair overall (lowest DP+EO):** **Post: EqOdds** (0.502), but the gain comes from raising female FPR.
- **If prioritizing recall parity (EO):** **Pre: Reweigh** is the best compromise (EO **0.135**, DP **0.391**).
- **Baseline** gives the best accuracy, but at the cost of the highest DP disparity.

---

First fairness mitigation: pre- and post-processing was performed on the designated best performing models (KNN, DT, RF, MLP) for CVD prediction.  In addition, these results are compared to a fairness-aware in-processing model - Adversarial Debiasing offered by AIF360.

In [22]:
#Adversarial Debiasing - In-processing by AIF360
import warnings
warnings.filterwarnings("ignore", category=FutureWarning)

try:
    import tensorflow as tf
    from aif360.algorithms.inprocessing import AdversarialDebiasing

    # TF1 graph mode - required by AIF360's implementation 
    tf.compat.v1.disable_eager_execution()
    tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.ERROR)

    # Build AIF360 datasets with FEATURES + label + sensitive attribute
    bld_tr = BinaryLabelDataset(
        df=pd.concat([
            pd.DataFrame(X_train_ready).reset_index(drop=True),
            pd.Series(y_train, name=label_name),
            pd.Series(A_train, name=protected_attr)
        ], axis=1),
        label_names=[label_name],
        protected_attribute_names=[protected_attr],
        favorable_label=favorable_label,
        unfavorable_label=unfavorable_label
    )

    bld_te = BinaryLabelDataset(
        df=pd.concat([
            pd.DataFrame(X_test_ready).reset_index(drop=True),
            pd.Series(y_test, name=label_name),
            pd.Series(A_test, name=protected_attr)
        ], axis=1),
        label_names=[label_name],
        protected_attribute_names=[protected_attr],
        favorable_label=favorable_label,
        unfavorable_label=unfavorable_label
    )

    # Train + predict inside a TF1 session
    sess = tf.compat.v1.Session()
    with sess.as_default():
        adv = AdversarialDebiasing(
            privileged_groups=privileged_groups,
            unprivileged_groups=unprivileged_groups,
            scope_name="adv_debias",
            debias=True,
            sess=sess
        )
        adv.fit(bld_tr)
        pred_te = adv.predict(bld_te)

        # Extract labels and (if available) scores
        yhat_adv = pred_te.labels.ravel().astype(int)
        scores_adv = getattr(pred_te, "scores", None)
        if scores_adv is None:
            scores_adv = yhat_adv.astype(float)

    # Clean up TF graph
    tf.compat.v1.reset_default_graph()
    sess.close()

    # Same structured output as other models
    _ = report_model(
        "ADV in-proc (AIF360)",
        y_test, yhat_adv, A_test,
        scores=scores_adv,
        note="trained on X_train_ready"
    )

except Exception as e:
    print("AdversarialDebiasing skipped:", type(e).__name__, e)



epoch 0; iter: 0; batch classifier loss: 0.652781; batch adversarial loss: 0.695995
epoch 1; iter: 0; batch classifier loss: 0.603601; batch adversarial loss: 0.689310
epoch 2; iter: 0; batch classifier loss: 0.580917; batch adversarial loss: 0.695957
epoch 3; iter: 0; batch classifier loss: 0.525679; batch adversarial loss: 0.700005
epoch 4; iter: 0; batch classifier loss: 0.471380; batch adversarial loss: 0.692137
epoch 5; iter: 0; batch classifier loss: 0.479058; batch adversarial loss: 0.696620
epoch 6; iter: 0; batch classifier loss: 0.454640; batch adversarial loss: 0.698632
epoch 7; iter: 0; batch classifier loss: 0.459749; batch adversarial loss: 0.697794
epoch 8; iter: 0; batch classifier loss: 0.388650; batch adversarial loss: 0.701929
epoch 9; iter: 0; batch classifier loss: 0.476035; batch adversarial loss: 0.696708
epoch 10; iter: 0; batch classifier loss: 0.435600; batch adversarial loss: 0.702427
epoch 11; iter: 0; batch classifier loss: 0.389775; batch adversarial los

,TPR,FPR,Recall,SelectionRate,Accuracy
Sex,,,,,
0,0.833333,0.125,0.833333,0.236842,0.868421
1,0.864583,0.160,0.864583,0.623288,0.856164


Overall -> Accuracy: 0.8587 | DP diff: 0.3864 | EO diff: 0.0312 | trained on X_train_ready


## ADV In-processing (AIF360)

### Results overview
| Variant        | Accuracy | DP diff | EO diff (TPR gap) | DP+EO |
|----------------|---------:|--------:|------------------:|------:|
| ADV in-proc    | 0.8587   | 0.3864  | **0.0312**        | **0.4176** |

---

### Per-group behavior (Female → 0, Male → 1)

#### ADV in-proc
- **Selection rate:** 0 **0.237**, 1 **0.623** → DP gap **0.386** (large).  
- **TPR (Recall):** 0 **0.833**, 1 **0.865** → **near parity** (EO gap **0.031**, excellent).  
- **FPR:** 0 **0.125**, 1 **0.160** (slightly higher for males).  
- **Accuracy:** 0 **0.868**, 1 **0.856** → very close per-group accuracy.  

---

### Implications
- **EO gap is minimal (0.031)**, meaning the model achieves almost perfect equality in true positive rates.  
- **DP gap remains large (0.386)**, so males are still recommended/selected much more often.  
- **Overall accuracy (0.859)** is competitive with your other models.  
- **Interpretation:** Adversarial Debiasing is highly effective at equalizing recall across groups but does not fully close the selection rate disparity.

---

In [23]:
# Grid-tune AIF360 AdversarialDebiasing for better DP/EO balance and print with report_model
import warnings
warnings.filterwarnings("ignore", category=FutureWarning)

import tensorflow as tf
from aif360.algorithms.inprocessing import AdversarialDebiasing

# small search over key knobs; widen if needed
ADV_GRID = dict(
    adversary_loss_weight=[0.02, 0.05, 0.1, 0.2, 0.3],
    num_epochs=[40, 60, 80],
    batch_size=[64, 128],
    classifier_num_hidden_units=[32, 64]  # size of main net
)

def run_adv(loss_w=0.1, epochs=50, bs=128, hidden=64, seed=42):
    tf.compat.v1.reset_default_graph()
    tf.compat.v1.disable_eager_execution()
    tf.compat.v1.set_random_seed(seed)
    sess = tf.compat.v1.Session()
    with sess.as_default():
        adv = AdversarialDebiasing(
            privileged_groups=privileged_groups,
            unprivileged_groups=unprivileged_groups,
            debias=True,
            scope_name=f"adv_w{loss_w}_e{epochs}_b{bs}_h{hidden}",
            adversary_loss_weight=loss_w,
            num_epochs=epochs,
            batch_size=bs,
            classifier_num_hidden_units=hidden,
            sess=sess
        )
        adv.fit(bld_tr)
        pred_te = adv.predict(bld_te)
        yhat = pred_te.labels.ravel().astype(int)
        scores = getattr(pred_te, "scores", None)
        if scores is None:
            scores = yhat.astype(float)
    sess.close()
    tf.compat.v1.reset_default_graph()
    return yhat, scores

# Build once (as you did)
bld_tr = BinaryLabelDataset(
    df=pd.concat([pd.DataFrame(X_train_ready).reset_index(drop=True),
                  pd.Series(y_train, name=label_name),
                  pd.Series(A_train, name=protected_attr)], axis=1),
    label_names=[label_name], protected_attribute_names=[protected_attr],
    favorable_label=favorable_label, unfavorable_label=unfavorable_label
)
bld_te = BinaryLabelDataset(
    df=pd.concat([pd.DataFrame(X_test_ready).reset_index(drop=True),
                  pd.Series(y_test, name=label_name),
                  pd.Series(A_test, name=protected_attr)], axis=1),
    label_names=[label_name], protected_attribute_names=[protected_attr],
    favorable_label=favorable_label, unfavorable_label=unfavorable_label
)

# Search & pick the best by minimizing (DP + EO) with an accuracy floor
best = None
acc_floor = 0.86  # keep close to your current accuracy; adjust as you like
results = []
for w in ADV_GRID["adversary_loss_weight"]:
    for e in ADV_GRID["num_epochs"]:
        for bs in ADV_GRID["batch_size"]:
            for h in ADV_GRID["classifier_num_hidden_units"]:
                yhat, scores = run_adv(w, e, bs, h)
                acc = accuracy_score(y_test, yhat)
                dp, eo = fair_metrics(y_test, yhat, A_test, scores, absolute=True)
                obj = dp + eo
                results.append((obj, acc, dp, eo, w, e, bs, h, yhat, scores))
                if (best is None or obj < best[0]) and acc >= acc_floor:
                    best = (obj, acc, dp, eo, w, e, bs, h, yhat, scores)

# Report best and (optionally) a few runners-up
if best is None:
    # fallback: take global best even if below floor
    best = sorted(results, key=lambda t: t[0])[0]

obj, acc, dp, eo, w, e, bs, h, yhat_best, scores_best = best
_ = report_model(
    f"ADV in-proc (best) w={w}, e={e}, b={bs}, h={h}",
    y_test, yhat_best, A_test, scores=scores_best,
    note=f"combined gap (DP+EO)={obj:.4f}; acc={acc:.4f}"
)

epoch 0; iter: 0; batch classifier loss: 0.679883; batch adversarial loss: 0.736842
epoch 1; iter: 0; batch classifier loss: 0.674382; batch adversarial loss: 0.737473
epoch 2; iter: 0; batch classifier loss: 0.637465; batch adversarial loss: 0.765090
epoch 3; iter: 0; batch classifier loss: 0.578616; batch adversarial loss: 0.654090
epoch 4; iter: 0; batch classifier loss: 0.549959; batch adversarial loss: 0.696969
epoch 5; iter: 0; batch classifier loss: 0.584526; batch adversarial loss: 0.703984
epoch 6; iter: 0; batch classifier loss: 0.523797; batch adversarial loss: 0.691683
epoch 7; iter: 0; batch classifier loss: 0.451424; batch adversarial loss: 0.807533
epoch 8; iter: 0; batch classifier loss: 0.382396; batch adversarial loss: 0.666175
epoch 9; iter: 0; batch classifier loss: 0.607451; batch adversarial loss: 0.787192
epoch 10; iter: 0; batch classifier loss: 0.482677; batch adversarial loss: 0.766742
epoch 11; iter: 0; batch classifier loss: 0.394712; batch adversarial loss:

,TPR,FPR,Recall,SelectionRate,Accuracy
Sex,,,,,
0,0.833333,0.09375,0.833333,0.210526,0.894737
1,0.843750,0.12000,0.843750,0.595890,0.856164


Overall -> Accuracy: 0.8641 | DP diff: 0.3854 | EO diff: 0.0104 | combined gap (DP+EO)=0.3958; acc=0.8641


## ADV In-processing (tuned)

### Results overview
| Variant            | Accuracy | DP diff | EO diff (TPR gap) | DP+EO |
|--------------------|---------:|--------:|------------------:|------:|
| ADV in-proc (best) | 0.8641   | 0.3854  | **0.0104**        | **0.3958** |

---

### Per-group behavior (Female → 0, Male → 1)

#### ADV in-proc (best, w=0.02, e=40, b=128, h=64)
- **Selection rate:** 0 **0.211**, 1 **0.596** → DP gap **0.385** (large).  
- **TPR (Recall):** 0 **0.833**, 1 **0.844** → **near-perfect parity** (EO gap **0.010**, excellent).  
- **FPR:** 0 **0.094**, 1 **0.120** (slightly higher for males).  
- **Accuracy:** Female **0.895**, Male **0.856** → both groups perform well.  

---

### Implications
- **EO gap is almost eliminated (0.010)** → the model equalizes recall across sexes very effectively.  
- **DP gap remains high (0.385)** → males are still selected more often than females.  
- **Accuracy (0.864)** is strong, slightly higher than the untuned ADV run.  
- **Overall:** Tuning improved both **accuracy** and **EO fairness**, while DP disparity remains the main limitation.

---

## Overall Comparison of Bias Mitigation Results

| Model / Variant         | Accuracy | DP diff | EO diff | DP+EO | Notes                                                                 |
|--------------------------|---------:|--------:|--------:|------:|-----------------------------------------------------------------------|
| **KNN – Baseline**       | 0.8750   | 0.4380  | 0.1875  | 0.6255 | High accuracy, large gaps.                                            |
| KNN – Pre: Reweigh       | 0.8315   | 0.3580  | **0.0417** | **0.3997** | Best EO; improved DP; accuracy drop.                                  |
| KNN – Post: EqOdds       | 0.8750   | 0.4380  | 0.1875  | 0.6255 | No real change from baseline.                                         |
| **DT – Baseline**        | 0.8207   | 0.3306  | **0.1042** | **0.4348** | Decent balance; EO best here.                                         |
| DT – Pre: Reweigh        | 0.7391   | **0.1968** | 0.1771 | 0.3739 | Best DP; worsened EO + accuracy.                                      |
| DT – Post: EqOdds        | 0.8261   | 0.3064  | 0.1667  | 0.4731 | Accuracy ↑; DP improved; EO worse.                                    |
| **RF – Baseline**        | 0.8315   | 0.3464  | **0.0000** | **0.3464** | Perfect EO; large DP; overall best balance for RF.                    |
| RF – Pre: Reweigh        | 0.8315   | 0.3464  | **0.0000** | **0.3464** | Identical to baseline.                                                |
| RF – Post: EqOdds        | 0.8261   | 0.3533  | **0.0000** | 0.3533 | Slight accuracy drop, DP worse.                                       |
| **MLP – Baseline**       | 0.8533   | 0.4243  | 0.1562  | 0.5805 | High accuracy, large gaps.                                            |
| MLP – Pre: Reweigh       | 0.8315   | 0.3911  | **0.1354** | 0.5265 | EO improved; DP somewhat better; accuracy drop.                       |
| MLP – Post: EqOdds       | 0.8370   | **0.3453** | 0.1562  | **0.5015** | Best DP for MLP; EO unchanged; accuracy between baseline & reweigh.   |
| **ADV in-proc**          | 0.8587   | 0.3864  | 0.0312  | 0.4176 | Excellent EO; DP still high.                                          |
| **ADV in-proc (tuned)**  | 0.8641   | 0.3854  | **0.0104** | **0.3958** | Best EO overall; accuracy strong; DP gap persists.                    |

---

## Key Takeaways
- **Best EO (true positive parity):** RF Baseline (0.000) and ADV tuned (0.010) → both nearly eliminate recall disparity.
- **Best DP (selection parity):** DT Pre: Reweigh (0.197) → but accuracy suffers.
- **Best combined fairness (DP+EO):** DT Pre (0.374) and ADV tuned (0.396) → different trade-offs.
- **Highest accuracy with fairness gains:** ADV tuned (0.864, EO ~0.01).
- **Models most resistant to mitigation:** RF – baseline already optimal, other methods add little.

 Overall, **Adversarial Debiasing (tuned)** gave the strongest fairness improvement on EO while keeping accuracy high, though DP disparity remains unresolved.
